In [54]:
pip install pandas scikit-learn numpy



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [55]:
pip install matplotlib



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from urllib.parse import urlparse
import numpy as np
from math import log2

In [57]:
import matplotlib.pyplot as plt

# Read the data

In [63]:
df = pd.read_csv('combined_df.csv')
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df

,url,label
0,www.usaozsazps.com/information,1
1,awrs.cl/wp-content/themes/form/bill.charged.html,1
2,ipfs.eth.aragon.network/ipfs/bafybeifg3yzh6ekg...,1
3,wwxhajudjgwjklckvzgs7.firebaseapp.com/,1
4,roomclip.jp,0
...,...,...
91721,optikgeneral.com/Webmail/12/Webmail/webmail.php,1
91722,thegioididong.com,0
91723,pressks.com,0
91724,68.178.168.47/Empresarial/,1


In [47]:
pip install tldextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 2.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [107]:
import pandas as pd
from urllib.parse import urlparse
import re

# Helper function to calculate the number of times each special character appears in the URL
def count_chars(url, char_list):
    return {f"num_{char}_url": url.count(char) for char in char_list}

# Function to check if an email is present in the URL
def check_email_in_url(url):
    return 1 if "@" in url else 0

# Calculate the length of the top-level domain (TLD)
def tld_length(url):
    domain = urlparse(url).netloc
    try:
        return len(domain.split('.')[-1])  # Assuming the last part after the last dot is the TLD
    except IndexError:
        return 0

# Function to extract features from URLs in a DataFrame
def extract_features_from_urls(df):
    def get_features(url):
        special_chars = ['.', '-', '_', '/', '?', '=', '@', '&', '!', ' ', '~', ',', '+', '*', '#', '$', '%']
        features = count_chars(url, special_chars)
        features['length_url'] = len(url)
        features['email_in_url'] = check_email_in_url(url)
        features['num_tld_url'] = tld_length(url)
        return features
    
    # Apply the get_features function to each URL in the DataFrame
    features_df = df['url'].apply(lambda x: pd.Series(get_features(x)))
    return pd.concat([df, features_df], axis=1)

# Function to extract domain features and process a DataFrame
def features_from_domain(df):
    def domain_features(url):
        domain = urlparse(url).netloc
        features = {
            'num_dots_dom': domain.count('.'),
            'num_hyph_dom': domain.count('-'),
            'num_undeline_dom': domain.count('_'),
            'num_vowels_dom': sum(1 for char in domain if char in 'aeiouAEIOU'),
            'length_dom': len(domain),
            'dom_in_ip': 1 if re.match(r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$", domain) else 0,
            'server_client_dom': 1 if 'server' in domain or 'client' in domain else 0
        }
        return features

    # Apply the domain_features function to each URL in the DataFrame
    features_df = df['url'].apply(lambda x: pd.Series(domain_features(x)))
    return pd.concat([df, features_df], axis=1)



In [108]:
#First, apply URL features extraction
df_features = extract_features_from_urls(df)

#Then, apply domain features extraction on the same DataFrame
df_features = features_from_domain(df_features)

print(df_features)

                                                     url  label  num_._url  \
0                         www.usaozsazps.com/information      1          2   
1       awrs.cl/wp-content/themes/form/bill.charged.html      1          3   
2      ipfs.eth.aragon.network/ipfs/bafybeifg3yzh6ekg...      1          3   
3                 wwxhajudjgwjklckvzgs7.firebaseapp.com/      1          2   
4                                            roomclip.jp      0          1   
...                                                  ...    ...        ...   
91721    optikgeneral.com/Webmail/12/Webmail/webmail.php      1          2   
91722                                  thegioididong.com      0          1   
91723                                        pressks.com      0          1   
91724                         68.178.168.47/Empresarial/      1          3   
91725                                usps.comhelpom.com/      1          2   

       num_-_url  num___url  num_/_url  num_?_url  num_=_url  n

In [109]:
def extract_pathname_features(url):
    """
    Extracts features from the pathname of a URL.
    """
    pathname = urlparse(url).path
    
    features = {
        'num_dots_path': pathname.count('.'),
        'num_hyph_path': pathname.count('-'),
        'num_undeline_path': pathname.count('_'),
        'num_slash_path': pathname.count('/'),
        'num_equal_path': pathname.count('='),
        'at_sign_path': pathname.count('@'),
        'num_and_path': pathname.count('&'),
        'num_exclamation_path': pathname.count('!'),
        'num_space_path': pathname.count(' '),
        'tilde_path': pathname.count('~'),
        'num_comma_path': pathname.count(','),
        'num_plus_path': pathname.count('+'),
        'num_asterisk_path': pathname.count('*'),
        'num_percent_path': pathname.count('%'),
        'length_path': len(pathname)
    }
    
    return features

def features_from_pathname(df):
    """
    Applies the extract_pathname_features function to each URL in a DataFrame and returns the DataFrame with added features.
    """
    # Apply the extract_pathname_features function to each URL in the DataFrame
    features_df = df['url'].apply(lambda x: pd.Series(extract_pathname_features(x)))
    return pd.concat([df, features_df], axis=1)


In [110]:
df_features = features_from_pathname(df_features)

print(df_features)

                                                     url  label  num_._url  \
0                         www.usaozsazps.com/information      1          2   
1       awrs.cl/wp-content/themes/form/bill.charged.html      1          3   
2      ipfs.eth.aragon.network/ipfs/bafybeifg3yzh6ekg...      1          3   
3                 wwxhajudjgwjklckvzgs7.firebaseapp.com/      1          2   
4                                            roomclip.jp      0          1   
...                                                  ...    ...        ...   
91721    optikgeneral.com/Webmail/12/Webmail/webmail.php      1          2   
91722                                  thegioididong.com      0          1   
91723                                        pressks.com      0          1   
91724                         68.178.168.47/Empresarial/      1          3   
91725                                usps.comhelpom.com/      1          2   

       num_-_url  num___url  num_/_url  num_?_url  num_=_url  n

In [112]:
def extract_filename_features(url):
    """
    Extracts features from the filename of a URL.
    """
    # Parse the URL to get the path
    path = urlparse(url).path
    # Extract the filename (assuming the filename is the last part of the path)
    filename = path.split('/')[-1] if '/' in path else path
    
    features = {
        'num_dots_file': filename.count('.'),
        'num_hyph_file': filename.count('-'),
        'num_undeline_file': filename.count('_'),
        'num_equal_file': filename.count('='),
        'at_sign_file': filename.count('@'),
        'num_and_file': filename.count('&'),
        'num_exclamation_file': filename.count('!'),
        'num_space_file': filename.count(' '),
        'tilde_file': filename.count('~'),
        'num_comma_file': filename.count(','),
        'num_plus_file': filename.count('+'),
        'num_percent_file': filename.count('%'),
        'length_file': len(filename)
    }
    
    return features

def features_from_filename(df):
    """
    Applies the extract_filename_features function to each URL in a DataFrame and returns the DataFrame with added features.
    """
    # Apply the extract_filename_features function to each URL in the DataFrame
    features_df = df['url'].apply(lambda x: pd.Series(extract_filename_features(x)))
    return pd.concat([df, features_df], axis=1)

In [113]:
df_features = features_from_filename(df_features)

print(df_features)

                                                     url  label  num_._url  \
0                         www.usaozsazps.com/information      1          2   
1       awrs.cl/wp-content/themes/form/bill.charged.html      1          3   
2      ipfs.eth.aragon.network/ipfs/bafybeifg3yzh6ekg...      1          3   
3                 wwxhajudjgwjklckvzgs7.firebaseapp.com/      1          2   
4                                            roomclip.jp      0          1   
...                                                  ...    ...        ...   
91721    optikgeneral.com/Webmail/12/Webmail/webmail.php      1          2   
91722                                  thegioididong.com      0          1   
91723                                        pressks.com      0          1   
91724                         68.178.168.47/Empresarial/      1          3   
91725                                usps.comhelpom.com/      1          2   

       num_-_url  num___url  num_/_url  num_?_url  num_=_url  n

In [114]:
from urllib.parse import urlparse, parse_qs
import pandas as pd

def extract_parameters_features(url):
    """
    Extracts features from the parameters of a URL.
    """
    query_string = urlparse(url).query
    params = parse_qs(query_string)
    
    # Counting the occurrences of various symbols in the entire query string
    features = {
        'num_dots_param': query_string.count('.'),
        'num_hyph_param': query_string.count('-'),
        'num_undeline_param': query_string.count('_'),
        'num_slash_param': query_string.count('/'),
        'num_questionmark_param': query_string.count('?'),
        'num_equal_param': query_string.count('='),
        'at_sign_param': query_string.count('@'),
        'num_and_param': query_string.count('&'),
        'num_exclamation_param': query_string.count('!'),
        'num_space_param': query_string.count(' '),
        'tilde_param': query_string.count('~'),
        'num_comma_param': query_string.count(','),
        'num_plus_param': query_string.count('+'),
        'num_asterisk_param': query_string.count('*'),
        'num_dollar_param': query_string.count('$'),
        'num_percent_param': query_string.count('%'),
        'length_param': len(query_string),
        'tld_in_param': int(any(part.endswith(('.com', '.org', '.net', '.edu', '.gov')) for part in query_string.split('&'))),
        'num_param': len(params)
    }
    
    return features

def features_from_parameters(df):
    """
    Applies the extract_parameters_features function to each URL in a DataFrame and returns the DataFrame with added features.
    """
    # Apply the extract_parameters_features function to each URL in the DataFrame
    features_df = df['url'].apply(lambda x: pd.Series(extract_parameters_features(x)))
    return pd.concat([df, features_df], axis=1)

In [115]:
df_features = features_from_parameters(df_features)

print(df_features)

                                                     url  label  num_._url  \
0                         www.usaozsazps.com/information      1          2   
1       awrs.cl/wp-content/themes/form/bill.charged.html      1          3   
2      ipfs.eth.aragon.network/ipfs/bafybeifg3yzh6ekg...      1          3   
3                 wwxhajudjgwjklckvzgs7.firebaseapp.com/      1          2   
4                                            roomclip.jp      0          1   
...                                                  ...    ...        ...   
91721    optikgeneral.com/Webmail/12/Webmail/webmail.php      1          2   
91722                                  thegioididong.com      0          1   
91723                                        pressks.com      0          1   
91724                         68.178.168.47/Empresarial/      1          3   
91725                                usps.comhelpom.com/      1          2   

       num_-_url  num___url  num_/_url  num_?_url  num_=_url  n

In [116]:
X = df_features.drop(['url', 'label'], axis=1)
y = df_features['label']


# Assuming X is your features and y is your labels
# Split the data into training and test sets with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# If you need a validation set as well
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)  # 0.25 x 0.8 = 0.2

# Feature selection
forest = RandomForestClassifier(n_estimators=100, random_state=42)
forest.fit(X_train, y_train)

selector = SelectFromModel(forest, prefit=True)
X_important_train = selector.transform(X_train)
X_important_val = selector.transform(X_val)
X_important_test = selector.transform(X_test)

/usr/local/python/3.10.13/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/python/3.10.13/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/python/3.10.13/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [117]:
# Get a mask, or integer index, of the features selected
selected_features_mask = selector.get_support()

# Now, print the names of the features selected
selected_feature_names = X_train.columns[selected_features_mask]
print("Selected Features:")
print(selected_feature_names.tolist())

Selected Features:
['num_._url', 'num_-_url', 'num_/_url', 'length_url', 'num_dots_path', 'num_hyph_path', 'num_slash_path', 'length_path', 'num_dots_file', 'length_file']


In [118]:
# Classification using selected features
forest_important = RandomForestClassifier(n_estimators=100, random_state=42)
forest_important.fit(X_important_train, y_train)





RandomForestClassifier(random_state=42)

In [119]:
from sklearn.metrics import classification_report, accuracy_score
# Validation prediction and evaluation
y_val_pred = forest_important.predict(X_important_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy}")

Validation Accuracy: 0.9899155083128918


In [120]:
# Predictions and evaluation selected
y_pred = forest_important.predict(X_important_test)
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.98      1.00      0.99      9200
           1       1.00      0.98      0.99      9146

    accuracy                           0.99     18346
   macro avg       0.99      0.99      0.99     18346
weighted avg       0.99      0.99      0.99     18346

